 1. Multinomial Logistic Regression
 2. Model & Interpretation
 3. Misclassification Error and Confusion Matrix
 4. Prediction and model assessment

<font color = 'blue'>
Content: 

1. [Multinomial Logistic Regression](#1)
   * 1.1 [Data](#2)
   * 1.2 [Data Partition](#3)
   * 1.3 [Multinominal Logistic Regression](#4)

1. [Model & Interpretation](#5)
    * 2.1. [Two Tail Z-test p-value](#6)
    * 2.2. [Second Model](#7)
1. [Misclassification Error and Confusion Matrix](#8)
    * 3.1. [Training Data](#9)
    * 3.2. [Testing Data](#10)
1. [Prediction and Model Assessment](#11)
1. [References](#12)



<a id = "1"></a><br>
# 1. Multinomial Logistic Regression
<a id = "2"></a><br>
# 1.1. Data

In [1]:
# Data
library (readr)

urlfile="https://raw.githubusercontent.com/bkrai/R-files-from-YouTube/main/Cardiotocographic.csv"
mydata<-read_csv(url(urlfile))
head(mydata)

# except NSP all variables are independent variables
# NSP is a dependent variable
## NSP = 1 --> Normal patient
## NSP = 2 --> Suspect patient
## NSP = 3 --> Pathologic patient
## N - Normal 
## S - Suspect
## P - Pathologic

Rows: 2126 Columns: 22

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (22): LB, AC, FM, UC, DL, DS, DP, ASTV, MSTV, ALTV, MLTV, Width, Min, Ma...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



LB,AC,FM,UC,DL,DS,DP,ASTV,MSTV,ALTV,⋯,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,NSP
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
120,0.000000000,0,0.000000000,0.000000000,0,0.000000000,73,0.5,43,⋯,62,126,2,0,120,137,121,73,1,2
132,0.006379585,0,0.006379585,0.003189793,0,0.000000000,17,2.1,0,⋯,68,198,6,1,141,136,140,12,0,1
133,0.003322259,0,0.008305648,0.003322259,0,0.000000000,16,2.1,0,⋯,68,198,5,1,141,135,138,13,0,1
134,0.002560819,0,0.007682458,0.002560819,0,0.000000000,16,2.4,0,⋯,53,170,11,0,137,134,137,13,1,1
132,0.006514658,0,0.008143322,0.000000000,0,0.000000000,16,2.4,0,⋯,53,170,9,0,137,136,138,11,1,1
134,0.001049318,0,0.010493179,0.009443861,0,0.002098636,26,5.9,0,⋯,50,200,5,3,76,107,107,170,0,3


In [2]:
# NSP is integer. We first need to convert from integer to factor
mydata$NSP <- as.factor(mydata$NSP)

<a id = "3"></a><br>
# 1.2. Data Partition
* CTG data
* Categorical response variables at three levels
* Data partition
* Multinomial Logistic Regression Model

In [3]:
# Data Partition
set.seed(222)
ind <- sample(2, nrow(mydata),
              replace = TRUE, # sampling for replacing
              prob = c(0.6, 0.4)) # we use probability for splitting


training <- mydata[ind == 1,  ]  # first data set we call training, 1 is first 60% of data
testing <- mydata[ind == 2, ] 
# training data has 1277 observations
# testing data has 849 observations

<a id = "4"></a><br>
## 1.3. Multinominal Logistic Regression

In [4]:
# Multinominal Logistic Regression
library(nnet)
training$NSP <- relevel(training$NSP, ref = "1") # as as reference we put 1 for normal patient
mymodel <- multinom(NSP~., data = training)    # it means all other variables, remember NSP is a dependent variable, apart from NSP we have 21 variables
# this is based on training data model
summary(mymodel)

# weights:  69 (44 variable)
initial  value 1402.927893 
iter  10 value 539.773236
iter  20 value 505.865352
iter  30 value 392.508097
iter  40 value 351.485743
iter  50 value 346.813302
iter  60 value 316.596959
iter  70 value 285.825391
iter  80 value 284.436641
iter  90 value 268.916989
iter 100 value 268.916331
final  value 268.916331 
stopped after 100 iterations


Call:
multinom(formula = NSP ~ ., data = training)

Coefficients:
  (Intercept)         LB          AC       FM        UC        DL          DS
2    -18.6761 -0.1143064 -1109.42102 16.34689 -300.4571 -73.95458 -0.73744258
3    -21.0584  0.4499481   -50.02172 19.36992 -384.2860 -76.86968 -0.04088559
        DP       ASTV        MSTV       ALTV         MLTV       Width
2 82.13684 0.08339589  0.01668685 0.02918356 -0.005775719 0.005367618
3 77.83336 0.20193024 -1.24041820 0.07712771  0.056655347 0.010265594
         Min        Max        Nmax      Nzeros        Mode       Mean
2 0.01840666 0.02377429  0.08924204 -0.21841801 -0.07898500  0.2157336
3 0.01462147 0.02488706 -0.13374794  0.09458505  0.05601353 -0.1114634
       Median   Variance  Tendency
2  0.02766913 0.05616717 0.2946502
3 -0.40033259 0.07035541 0.1481800

Std. Errors:
  (Intercept)         LB          AC        FM          UC          DL
2   2.2222232 0.03794180 0.014531690 2.1560902 0.003467970 0.001159880
3   0.5906376 0.

<a id = "5"></a><br>
# 2. Model and Interpretation
<a id = "6"></a><br>
## 2.1. Two Tail Z-test p-value

In [5]:
# Finalizing model
## Two Tail Z-test p-value
z <- summary(mymodel)$coefficients/summary(mymodel)$standard.errors
p <- pnorm(1 - pnorm(abs(z), 0, 1)) * 2  # we add 2 due two tail z-test
p

,(Intercept),LB,AC,FM,UC,DL,DS,DP,ASTV,MSTV,⋯,Width,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency
2,1,1.001033,1,1,1,1,1,1,1,1.368149,⋯,1.126625,1.023911,1.024127,1.074744,1.089249,1.004545,1.000247,1.285837,1,1.154310
3,1,1.000000,1,1,1,1,1,1,1,1.005114,⋯,1.071251,1.111865,1.028117,1.134181,1.323380,1.088421,1.008825,1.000000,1,1.301936


<a id = "7"></a><br>
## 2.2. Second Model

In [6]:
# SECOND MODEL
# we will not use the value more than 0.05 
# let's adjust model again
mymodel <- multinom(NSP~. -MLTV -Width - Min - Max - Nmax -Nzeros - Tendency,
                    data = training)


z <- summary(mymodel)$coefficients/summary(mymodel)$standard.errors
p <- pnorm(1 - pnorm(abs(z), 0, 1)) * 2  # we add 2 due two tail z-test
p

# weights:  48 (30 variable)
initial  value 1402.927893 
iter  10 value 524.667208
iter  20 value 396.280355
iter  30 value 363.851254
iter  40 value 355.488322
iter  50 value 313.735159
iter  60 value 290.981697
iter  70 value 290.876039
iter  80 value 290.786285
iter  90 value 282.315552
iter 100 value 279.633852
final  value 279.633852 
stopped after 100 iterations


,(Intercept),LB,AC,FM,UC,DL,DS,DP,ASTV,MSTV,ALTV,Mode,Mean,Median,Variance
2,1,1.009405,1,1,1,1,1,1,1,1.347954,1.000007,1.003839,1.001802,1.091561,1
3,1,1.000000,1,1,1,1,1,1,1,1.011089,1.000000,1.141510,1.018859,1.000002,1


In [7]:
# Interpretation
# Eqauation
# NSP = 2  patient is suspicious, NSP = 1 patient is normal, this log-odds
# we don't have y and nsp
# this is first equation
# In[P(NSP=2) / (NSP=1)] = -16,62047 + (-0.07164 * LB) + (-748.85498 * AC) + ....+ (0.0464* Varience)


# second equation
# NSP = 3 patient is Pathologic 
# In[P(NSP=3)/P(NSP=1)] = -18.55244 + (0.40854 * LB) + (-29.62735 * AC) + .... + (0.6643 / Varience)

<a id = "8"></a><br>
# 3. Confusion Matrix & Misclassification Error 
<a id = "9"></a><br>
## 3.1 Training Data

In [8]:
# Confusion Matrix & Misclassification Error - Training Data
p <- predict(mymodel, training)
head(p)

[1] 1 1 1 3 3 1
Levels: 1 2 3

In [9]:
head(training$NSP)
# when we compare the outputs, out of six predictions we see that 5 are correct, last one is wrong

[1] 1 1 1 3 3 3
Levels: 1 2 3

In [10]:
# top numbers of the table is actual numbers, prediction is on the left side
tab <- table(p, training$NSP)
tab

   
p     1   2   3
  1 965  57   7
  2  33 100  12
  3   6  12  85

In [11]:
# accuracy
sum(diag(tab)) / sum(tab)

[1] 0.9005482

In [12]:
# misclassifications
1 - sum(diag(tab)) / sum(tab)

[1] 0.09945184

<a id = "10"></a><br>
## 3.2 Testing Data

In [13]:
# Confusion Matrix & Misclassification Error - testing Data
p1 <- predict(mymodel, testing)
tab1 <- table(p1, testing$NSP)
tab1

   
p1    1   2   3
  1 619  36   4
  2  26  84  15
  3   6   6  53

In [14]:
# missclassifications
1 - sum(diag(tab1)) / sum(tab1)

[1] 0.1095406

<a id = "11"></a><br>
# 4. Prediction & Model Assessment

In [15]:
# Accuracy and Sensitivity - training data
n <- table(training$NSP)
# In training data set has 1004 patients are normal, 169 of them is suspucious, 104 Pathologic
# let's see them as a ratio
n / sum(n)


         1          2          3 
0.78621770 0.13234143 0.08144088 

In [16]:
tab / colSums(tab)
# this model 96 percent correct classification for 1, 59% correct for 2, 81% correct for 3
# we made a good job 1 and 3 
# 2 is not good as them as 

   
p             1           2           3
  1 0.961155378 0.056772908 0.006972112
  2 0.195266272 0.591715976 0.071005917
  3 0.057692308 0.115384615 0.817307692

In [17]:
tab1 / colSums(tab1)

   
p1            1           2           3
  1 0.950844854 0.055299539 0.006144393
  2 0.206349206 0.666666667 0.119047619
  3 0.083333333 0.083333333 0.736111111

<a id = "12"></a><br>
# 5. References
* https://www.youtube.com/watch?v=S2rZp4L_nXo&list=PL34t5iLfZddvv-L5iFFpd_P1jy_7ElWMG
* https://www.youtube.com/watch?v=oxRy2DMrOF4&list=PL34t5iLfZddvv-L5iFFpd_P1jy_7ElWMG&index=2
* https://www.youtube.com/watch?v=11VY8CmNVDQ&list=PL34t5iLfZddvv-L5iFFpd_P1jy_7ElWMG&index=3
* https://www.youtube.com/watch?v=POyTaeneHJY&list=PL34t5iLfZddvv-L5iFFpd_P1jy_7ElWMG&index=4